In [35]:
#pip install matplotlib

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [37]:
root_path = '/Users/lpwilson/Ada/Developer/projects/Capstone/EEO1s/'
#read in the long format csv
EEO1_data = pd.read_csv(root_path + 'long_csv.csv')

In [38]:
EEO1_data = pd.DataFrame(EEO1_data)
EEO1_data = EEO1_data.drop(columns = ['Unnamed: 0'])
#EEO1_data.info()
#EEO1_data

# goal: 
make a line plot filtered by company and job category that tracks gender (or race) 
as a percentage of total employees in that category over years. 

so, the x-axis points will be the year values
and the y-axis points will be the percentage of employees of each gender.

let's try to get the x- and y- values that we need. 

In [40]:
#set up the filters
company = "Amazon"
#I want to only look at some job categories for now, for readability.
job_cats = ['Exec/Sr. Officials & Mgrs', 'Administrative Support', 'Laborers & Helpers']
#choosing these names to match our queries.
sortBy1 = 'job_category'
sortBy2 = 'gender'

amz_data = EEO1_data.query(f"company == '{company}'")
# years = sorted(amz_data['year'].unique())
# years
amz_data

,count_employees,job_category,gender,race,year,company
560,106,Exec/Sr. Officials & Mgrs,Male,Hispanic or Latino,2021,Amazon
561,5192,First/Mid Officials & Mgrs,Male,Hispanic or Latino,2021,Amazon
562,4785,Professionals,Male,Hispanic or Latino,2021,Amazon
563,1240,Technicians,Male,Hispanic or Latino,2021,Amazon
564,9819,Sales Workers,Male,Hispanic or Latino,2021,Amazon
...,...,...,...,...,...,...
975,612,Administrative Support,Female,Two or More Races,2019,Amazon
976,3,Craft Workers,Female,Two or More Races,2019,Amazon
977,453,Operatives,Female,Two or More Races,2019,Amazon
978,6107,Laborers & Helpers,Female,Two or More Races,2019,Amazon


Now let's try to get the y-values for one of the job categories.
we want the sum of count_employees for a given job category and gender.

Let's also filter (query) by the job cats we want.

In [42]:

dbl_qry_amz = amz_data.query('job_category.isin(@job_cats)').groupby(["year", sortBy1, sortBy2])['count_employees'].sum()
dbl_qry_amz

year  job_category               gender
2019  Administrative Support     Female     11179
                                 Male        6539
      Exec/Sr. Officials & Mgrs  Female       475
                                 Male        1638
      Laborers & Helpers         Female    159556
                                 Male      151466
2020  Administrative Support     Female     13013
                                 Male        7781
      Exec/Sr. Officials & Mgrs  Female       629
                                 Male        1981
      Laborers & Helpers         Female    313982
                                 Male      308095
2021  Administrative Support     Female     13592
                                 Male        8075
      Exec/Sr. Officials & Mgrs  Female       792
                                 Male        2283
      Laborers & Helpers         Female    381350
                                 Male      380218
Name: count_employees, dtype: int64